In [2]:
%run 1-0mmyolo-common.ipynb

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Nov_30_19:08:53_PST_2020
Cuda compilation tools, release 11.2, V11.2.67
Build cuda_11.2.r11.2/compiler.29373293_0
gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-44)
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

Sun Feb 25 00:26:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                     

# Write custom Config file

In [3]:
# %cd {HOME}/mmyolo
%cd {HOME}/mmdetection

/work/van-speech-nlp/jindaznb/j-vis/mmdetection


In [4]:
dataset_location=f"/work/van-speech-nlp/jindaznb/j-vis/Forestfire2024-1-yuv"
model_name="custom-dynamic-rcnn_r50_fpn_1x_coco-yuv"
CUSTOM_CONFIG_PATH = f"{HOME}/mmdetection/configs/dynamic_rcnn/{model_name}.py"


CUSTOM_CONFIG = f"""
_base_ = '../faster_rcnn/faster-rcnn_r50_fpn_1x_coco.py'



# ========================Frequently modified parameters======================
# -----data related-----
model = dict(
    roi_head=dict(
        bbox_head=dict(num_classes=4),
))

dataset_type = 'COCODataset'
max_epochs = 200
classes =  ('Alive Tree', 'Beetle-Fire Tree', 'Dead Tree', 'Debris') 

data_root = "{dataset_location}" # Root directory of the dataset

train_ann_file = 'train/_annotations.coco.json'  # Annotation file for training set
train_data_prefix = 'train/'  # Prefix for training data directory

val_ann_file = 'valid/_annotations.coco.json'  # Annotation file for validation set
val_data_prefix = 'valid/'  # Prefix for validation data directory

class_name = ('Alive Tree', 'Beetle-Fire Tree', 'Dead Tree', 'Debris')  
num_classes = 4  # Number of classes in the dataset
metainfo = dict(classes=class_name, palette=[(20, 220, 60)])  # Metadata information for visualization

#train_batch_size_per_gpu = 8  # Batch size per GPU during training
#train_num_workers = 4  # Number of worker processes for data loading during training
persistent_workers = True  # Whether to use persistent workers during training

# -----train val related-----
# base_lr = 0.004  # Base learning rate for optimization
base_lr = 0.04
max_epochs = 200  # Maximum training epochs
num_epochs_stage2 = 20  # Number of epochs for stage 2 training

model_test_cfg = dict(
    multi_label=True,  # Multi-label configuration for multi-class prediction
    nms_pre=30000,  # Number of boxes before NMS
    score_thr=0.001,  # Score threshold to filter out boxes
    nms=dict(type='nms', iou_threshold=0.65),  # NMS type and threshold
    max_per_img=300)  # Maximum number of detections per image


# ========================Possible modified parameters========================
default_hooks = dict(
    checkpoint=dict(
        type="CheckpointHook",
        save_best="coco/bbox_mAP_50",
        rule="greater",
        max_keep_ckpts=10,
    ),
    early_stopping=dict(
        type="EarlyStoppingHook",
        monitor="coco/bbox_mAP_50",
        patience=15,
        min_delta=0.001
    ),
)

train_cfg=dict(
    max_epochs=max_epochs
)

data = dict(
    #samples_per_gpu=1,
    #workers_per_gpu=2,
    train=dict(
        type=dataset_type,
        img_prefix='train/',
        classes=classes,
        ann_file='train/_annotations.coco.json.json'),
    val=dict(
        type=dataset_type,
        img_prefix='valid/',
        classes=classes,
        ann_file='valid/_annotations.coco.json'),
    test=dict(
        type=dataset_type,
        img_prefix='test/',
        classes=classes,
        ann_file='test/_annotations.coco.json'))

# -----data related-----
img_scale = (1024, 1024)  # width, height
# ratio range for random resize
random_resize_ratio_range = (0.1, 2.0)
# Cached images number in mosaic
mosaic_max_cached_images = 40
# Number of cached images in mixupep
mixup_max_cached_images = 20
# Batch size of a single GPU during validation
val_batch_size_per_gpu = 8
# Worker to pre-fetch data for each single GPU during validation
val_num_workers = 3

# Config of batch shapes. Only on val.
batch_shapes_cfg = dict(
    type='BatchShapePolicy',
    batch_size=val_batch_size_per_gpu,
    img_size=img_scale[0],
    size_divisor=32,
    extra_pad_ratio=0.5)

# -----model related-----
# The scaling factor that controls the depth of the network structure
deepen_factor = 0.67
# The scaling factor that controls the width of the network structure
widen_factor = 0.75
# Strides of multi-scale prior box
strides = [8, 16, 32]

norm_cfg = dict(type='BN')  # Normalization config

# -----train val related-----
lr_start_factor = 1.0e-5
dsl_topk = 13  # Number of bbox selected in each level
loss_cls_weight = 1.0
loss_bbox_weight = 2.0
qfl_beta = 2.0  # beta of QualityFocalLoss
weight_decay = 0.05

# Save model checkpoint and validation intervals
save_checkpoint_intervals = 10 
# validation intervals in stage 2
val_interval_stage2 = 1
# The maximum checkpoints to keep.
max_keep_ckpts = 3
# single-scale training is recommended to
# be turned on, which can speed up training.
env_cfg = dict(cudnn_benchmark=True)
"""
with open(CUSTOM_CONFIG_PATH, 'w') as file:
    file.write(CUSTOM_CONFIG)

In [7]:
# # !python tools/train.py configs/ssd/costom_ssd512.py --resume | tee ../OUT_ssd.txt
!python tools/train.py configs/dynamic_rcnn/custom-dynamic-rcnn_r50_fpn_1x_coco-yuv.py --resume  | tee "../OUT_dyn_$(date +"%Y%m%d_%H%M%S").txt"

^C
Traceback (most recent call last):
  File "/work/van-speech-nlp/jindaznb/j-vis/mmdetection/tools/train.py", line 8, in <module>
    from mmengine.runner import Runner
  File "/work/van-speech-nlp/jenv/lib/python3.10/site-packages/mmengine/runner/__init__.py", line 2, in <module>
    from ._flexible_runner import FlexibleRunner
  File "/work/van-speech-nlp/jenv/lib/python3.10/site-packages/mmengine/runner/_flexible_runner.py", line 10, in <module>
    import torch.nn as nn
  File "/work/van-speech-nlp/jenv/lib/python3.10/site-packages/torch/__init__.py", line 1285, in <module>
    from torch import quantization as quantization
  File "/work/van-speech-nlp/jenv/lib/python3.10/site-packages/torch/quantization/__init__.py", line 1, in <module>
    from .quantize import *  # noqa: F403
  File "/work/van-speech-nlp/jenv/lib/python3.10/site-packages/torch/quantization/quantize.py", line 10, in <module>
    from torch.ao.quantization.quantize import _convert
  File "/work/van-speech-nlp/jen

In [8]:
dataset_location

'/work/van-speech-nlp/jindaznb/j-vis/Forestfire2024-1-yuv'

In [6]:
ds = sv.DetectionDataset.from_coco(
    images_directory_path=f"{dataset_location}/test",
    annotations_path=f"{dataset_location}/test/_annotations.coco.json",
)

images = list(ds.images.values())
CUSTOM_WEIGHTS_PATH = glob.glob(f"{HOME}/mmdetection/work_dirs/{model_name}/best_coco_bbox_mAP_50_epoch_*.pth")[-1]
model = init_detector(CUSTOM_CONFIG_PATH, CUSTOM_WEIGHTS_PATH, device=DEVICE)

best_map, best_params = optimize_params(ds, model, param_space)
print('Best mAP:', best_map)
print('Best Parameters:', best_params)

Loads checkpoint by local backend from path: /work/van-speech-nlp/jindaznb/j-vis/mmdetection/work_dirs/custom-dynamic-rcnn_r50_fpn_1x_coco-yuv/best_coco_bbox_mAP_50_epoch_189.pth
|   iter    |  target   | confid... | nms_io... |
-------------------------------------------------
| 1         | 0.04209   | 0.4371    | 0.9556    |
| 2         | 0.08368   | 0.7588    | 0.6388    |
| 3         | 0.04055   | 0.2404    | 0.2404    |
| 4         | 0.03693   | 0.1523    | 0.8796    |
| 5         | 0.06134   | 0.641     | 0.7373    |
| 6         | 0.07129   | 0.9129    | 0.5792    |
| 7         | 0.08045   | 0.7101    | 0.4667    |
| 8         | 0.06432   | 0.8691    | 0.1351    |
| 9         | 0.0       | 1.0       | 0.9538    |
| 10        | 0.06288   | 0.5899    | 0.1       |
| 11        | 0.0779    | 0.9953    | 0.3472    |
| 12        | 0.04453   | 0.5006    | 0.5108    |
| 13        | 0.05876   | 0.8354    | 0.3691    |
| 14        | 0.08243   | 0.7481    | 0.638     |
| 15        | 0.08554